I found seemingly interesting code [here](https://florianwilhelm.info/2016/03/jupyter_distribution_visualizer/) and tried to make it work. Unfortunately, I failed to make it dynamically update a Bokeh plot. Still want to keep it for some time.

In [7]:
import pandas as pd
import numpy as np
import scipy.stats as stats

from collections import OrderedDict, defaultdict
from enum import Enum
import math

from bokeh.io import output_notebook, show, push_notebook
from bokeh.plotting import figure
from bokeh.io import show
output_notebook()

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

Loading BokehJS ...

In [8]:
import traceback
import sys

In [15]:
# How to enable widgets:
# https://stackoverflow.com/a/38001920/564240
# !pip install ipywidgets
from ipywidgets import widgets, interact, interactive
from IPython.display import display

In [26]:
class DistType(Enum):
    continuous = 0
    discrete = 1
    
    def __str__(self):
        if self == DistType.continuous:
            return "Continuous"
        return "Discrete"
    
class Distribution:
        
    def __init__(self, name, stats_instance):
        self._name = name
        self.stats_instance = stats_instance       
        
    def name(self):
        return self._name
        
    def get_type(self):
        raise NotImplementedError
             
    def functions(self):
        raise NotImplementedError    
        
    def has_shape_params(self):
        return len(self.shape_params()) != 0     
    
    def shape_params(self):
        if self.stats_instance.shapes is not None:
            return self.stats_instance.shapes.split(', ')
        return []  
    
    def support(self, *shapeargs):
        # due to bug in scipy.levy_stable no keyword args for interval
        return self.stats_instance.interval(1.0, *shapeargs)
    
class ContinuesDistribution(Distribution):

    @staticmethod
    def _fuctions(): 
        return ['pdf', 'cdf', 'ppf']   
    
    @staticmethod    
    def create_all():
        dists = [getattr(stats, d) for d in sorted(dir(stats)) if isinstance(getattr(stats, d), stats.rv_continuous)]
        return {dist.name: ContinuesDistribution(dist.name, dist) for dist in dists}
    
    def __init__(self, name, stats_instance):
        super(ContinuesDistribution, self).__init__(name, stats_instance)
        
    def get_type(self):
        return DistType.continuous
    
    def functions(self):
        return ContinuesDistribution._fuctions()
    
    def func_xy(self, func, *shapeargs, **params):
        if func == 'ppf':
            interval = [np.float64(0.), np.float64(1.)]
        else:
            interval = list(self.support(*shapeargs))
                 
        for i, x in enumerate(interval):
            if np.isnan(x) or math.isnan(x) or np.isinf(x):
                interval[i] = np.sign(x)*100
            interval[i] += (-1)**i*1e-3
        
        l, r = interval
        l, r = round(l, 0).astype(int), round(r, 0).astype(int)
        x = np.linspace(l.astype(int), r.astype(int), 100*(r-l))        
        y = getattr(self.stats_instance, func)(x, *shapeargs, **params)
        return x, y   
    
    
class DiscreteDistribution(Distribution):
      
    @staticmethod
    def _fuctions():
        return ['pmf', 'cdf'] 

    @staticmethod    
    def create_all():
        dists = [getattr(stats, d) for d in sorted(dir(stats)) if isinstance(getattr(stats, d), stats.rv_discrete)]
        return {dist.name: DiscreteDistribution(dist.name, dist) for dist in dists}
        
    def __init__(self, name, stats_instance):
        super(DiscreteDistribution, self).__init__(name, stats_instance)        
        
    def get_type(self):
        return DistType.discrete    
    
    def functions(self):
        return DiscreteDistribution._fuctions()
    
    def func_xy(self, func, *shapeargs, **params):
        interval = list(self.support(*shapeargs))
        for i, x in enumerate(interval):
            if np.isnan(x):
                interval[i] = 1.0 # np.sign(x)*20
        l, r = interval
        x = np.arange(l+1, r+1)
        
        y = getattr(self.stats_instance, func)(x, *shapeargs, **params)
        return x, y       
    
    
class StatsDistributionModel:
    
    @staticmethod
    def _default_shapes():
        DEFAULT_SHAPES = defaultdict(lambda: defaultdict(lambda: 1.0))
        DEFAULT_SHAPES['alpha'] = {'a': 1.3}
        DEFAULT_SHAPES['beta'] = {'a': 1.5, 'b': 2.}
        # discrete
        DEFAULT_SHAPES['bernoulli'] = {'p': 0.7}
        DEFAULT_SHAPES['binom'] = {'n': 10, 'p': 0.7}
        DEFAULT_SHAPES['logser'] = {'p': 0.3}
        DEFAULT_SHAPES['zipf'] = {'a': 2}
        DEFAULT_SHAPES['randint'] = {'low': 0, 'high': 10}
        DEFAULT_SHAPES['nbinom'] = {'n': 10, 'p': 0.6}
        DEFAULT_SHAPES['hypergeom'] = {'n': 3, 'M': 10, 'N': 7}
        DEFAULT_SHAPES['geom'] = {'p': 0.6}
        
        return DEFAULT_SHAPES
        
    def __init__(self):
        self._continuous = ContinuesDistribution.create_all()
        self._discrete = DiscreteDistribution.create_all()    
        
    def continuous(self):
        return self._continuous
    
    def discrete(self):
        return self._discrete
    
    def default_shape_params(self, dist_name):
        defaults = StatsDistributionModel._default_shapes()
        return defaults.get(dist_name, {})
    
    def default_distribution(self):
        return self._continuous["norm"]
    
    def available_distribution_types(self):
        return [DistType.continuous, DistType.discrete]
    
    def distributions(self, dist_type):
        if dist_type is DistType.continuous:
            return self.continuous().keys()
        else:
            return self.discrete().keys()
        
    def distibution(self, dist_name):
        if self.continuous().get(dist_name, None) is not None:
            return self.continuous()[dist_name]
        return self.discrete()[dist_name]

class View:
    def __init__(self, model):
        self.model = model
        default_distribution = self.model.default_distribution()
        self._create_widgets(default_distribution)
        self._set_updates()
        
        display(self.container)
        display(self.distribution_parametors_container)
        display(self.shape_param_container)
        display(self.error_text)
        
        self.fig, self.l_data, self.p_data = self._generate_fig_data()
        self.on_distribution_update()
#        show(self.fig)
        
    def _create_widgets(self, default_distribution):
        self.type_selector = widgets.Dropdown(
            options=self.model.available_distribution_types(), 
            description='type:',
            value=default_distribution.get_type()
        )
        
        self.distribution_selector = widgets.Dropdown(
            options=self.model.distributions(default_distribution.get_type()), 
            description='distribution:',
            value=default_distribution.name()
        )
        
        self.function_selector = widgets.Dropdown(
            options=default_distribution.functions(), 
            description='function:',
            value=default_distribution.functions()[0]
        )
        self.container = widgets.HBox()  
        self.container.children = [
            self.type_selector, 
            self.distribution_selector, 
            self.function_selector
        ]
        
        self.loc_slider = widgets.FloatSlider(
            value=0., min=-5.0, max=5.0, 
            step=0.1, description='loc:'
        )
        self.scale_slider = widgets.FloatSlider(
            value=1., min=0.01, max=10.0, 
            step=0.01, description='scale:'
        )
        self.distribution_parametors_container = widgets.VBox()
        self.shape_param_container = widgets.HBox()
        self.error_text = widgets.HTML()
        
    def _set_updates(self):
        self.type_selector.on_trait_change(self.on_type_update, name='value')
        
        self.distribution_selector.on_trait_change(self.on_distribution_update, name='value')
        self.function_selector.on_trait_change(self.on_distribution_parameter_update, name='value')
        
        self.loc_slider.on_trait_change(self.on_distribution_parameter_update, name='value')
        self.scale_slider.on_trait_change(self.on_distribution_parameter_update, name='value')        
        
    def on_type_update(self):
        self.distribution_selector.options = self.model.distributions(self.type_selector.value)
  
    def on_distribution_parameter_update(self):     
        
        shapeargs = [c.value for c in self.shape_param_container.children]
        print("shapeargs: ", shapeargs)
        self.l_data['x'], self.l_data['y'] = [], []
        self.p_data['xs'], self.p_data['ys'] = [], []
        
        try:
            if self.type_selector.value is DistType.continuous:
                self.refresh_continuous(self.l_data, *shapeargs)
            else:
                self.refresh_discrete(self.p_data, *shapeargs)
        except Exception as e:
            print(traceback.format_exc())
            print("********************")
            print(sys.exc_info()[2])
            self.error_text.value = "Invalid parameters! Choose again.<br>ERROR: {}".format(e)
            self.error_text.visible = True
        else:
            self.error_text.value = ""
            self.error_text.visible = False
            
        show(self.fig, notebook_handle=True)
        push_notebook() 
        
    def on_distribution_update(self):
        distribution = self.model.distibution(self.distribution_selector.value)
        if distribution.has_shape_params():
            shapes = self.model.default_shape_params(distribution.name())
            
            text_inputs = [widgets.BoundedFloatText(value=v, description='{}:'.format(k)) for k, v in shapes.items()]
            [w.on_trait_change(self.on_distribution_parameter_update, name='value') for w in text_inputs]
            self.shape_param_container.children = text_inputs
        else:
            self.shape_param_container.children = []
            
        if self.type_selector.value is DistType.continuous:
            self.distribution_parametors_container.children = [self.loc_slider, self.scale_slider]
        else:
            self.distribution_parametors_container.children = []
   
        self.function_selector.options = distribution.functions()    

    def refresh_continuous(self, data, *shapeargs):
        distribution = self.model.distibution(self.distribution_selector.value)
        params = dict(loc=self.loc_slider.value, scale=self.scale_slider.value)
        data['x'], data['y'] = distribution.func_xy(
            self.function_selector.value, 
            *shapeargs, **params
        ) 
        
        self.fig.y_range.start = max(data['y'].max() - 5, 1.1*data['y'].min())
        self.fig.y_range.end = min(data['y'].min() + 5, 1.1*data['y'].max())
        offset, lim = 1e-1, 5
        self.fig.x_range.start = max(-lim, np.min(data['x']) - offset)
        self.fig.x_range.end = min(lim, np.max(data['x']) + offset)
          

    def refresh_discrete(self, data, *shapeargs):
        distribution = self.model.distibution(self.distribution_selector.value)
        x, y = distribution.func_xy(self.function_selector.value, *shapeargs)
        data['xs'], data['ys'] = self._make_patches(x, y)
        
        self.fig.y_range.start, self.fig.y_range.end = 0., 1.1*np.max(y)
        self.fig.x_range.start = max(-10, np.min(x) - 1)
        self.fig.x_range.end = min(10, np.max(x) + 1)    
    
    def _make_patches(self, x, y, width=0.8):
        m = width/2
        x = [[p-m, p-m, p+m, p+m] for p in x]
        y = [[0, p, p, 0] for p in y]
        return x, y

    def _generate_fig_data(self):
        fig = figure(width=700, height=700, title=None, x_range=(-1, 1), y_range=(0, 1))
        ren_p = fig.patches([[]], [[]], line_width=3, alpha=0.3)
        ren_l = fig.line([1., 2.], [1., 2.], line_width=3)
        return fig, ren_l.data_source.data, ren_p.data_source.data
  
        
view = View(StatsDistributionModel())        

VBox()

HBox()

HTML(value='')